##### Imports

In [1]:
import tensorflow as tf
import cv2
from object_detection.utils import config_util  
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_util
from object_detection.protos import pipeline_pb2
from object_detection.builders import model_builder
from google.protobuf import text_format
import numpy as np
import os

In [6]:
%cd ./RealTimeObjectDetection

C:\Users\ankit\Py-world\Computer Vision\TFOD\RealTimeObjectDetection


##### Path Setting 

In [2]:
workspace_path='Tensorflow/workspace'  #: workspace contains dataset, scripts, pre trained model, output model 
scripts_path='Tensorflow/scripts'   #: contain scripts to generate tf record 
annotation_path=workspace_path+'/annotations'  #: contains label for dataset
image_path=workspace_path+'/images'  #: train and test images for training the model
api_model='Tensorflow/models'       #: tfod api model path
model_path=workspace_path+'/models'  #: after training the model will be stored in model_path
pretrained_model_path=workspace_path+'/pre-trained-models'  #: ssd mobilnet pre trained model path
config_path=model_path+'/my_ssd_mobnet/pipeline.config'  #: config file for the pre trained model
checkpoint_path=model_path+'/my_ssd_mobnet'  #: check point for the model(load the model from the checkpoint last time the model was trained)

##### Generating the label map files

In [5]:
labels=[{'name':'mask','id':1},{'name':'no mask','id':2}]
with open(annotation_path+'/labelmap.pbtxt', 'w') as f:
    for label in labels:
            f.write('item { \n')
            f.write('\tname:\'{}\'\n'.format(label['name']))
            f.write('\tid:{}\n'.format(label['id']))
            f.write('}\n')

##### Generate TF Record for training and testing data

In [69]:
!python {scripts_path+'/generate_tfrecord.py'} -x{image_path+'/train'} -l {annotation_path+'/labelmap.pbtxt'} -o {annotation_path+ '/train.record'}
!python {scripts_path+'/generate_tfrecord.py'} -x{image_path+ '/test'}  -l {annotation_path+ '/labelmap.pbtxt'} -o {annotation_path + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


##### Creating a new folder inside the workspace --> models --> my_ssd_mobilet, where we will store our trained model, checkpoints and configs

In [26]:
custom_model='my_ssd_mobnet' 

In [ ]:
!mkdir {'Tensorflow\workspace\models\\'+custom_model}
!cp  {pretrained_model_path+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {model_path+'/'+custom_model}
#: copy the pipeline.config file from the pre trained model folder and paste the config file inside my_ssd_mobilnet

##### Opening the config file

In [74]:
config_path=model_path+'/'+custom_model+'/pipeline.config'
config=config_util.get_configs_from_pipeline_file(config_path)
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

##### Creating a empty configuratio object to save the config in that

In [71]:
pipeline_config=pipeline_pb2.TrainEvalPipelineConfig()  #: empty configuration object
with tf.io.gfile.GFile(config_path,'r') as f:           #: read the config file
    str=f.read()                                        #: stored the config in temporary object
    text_format.Merge(str,pipeline_config)              #: stored the object in pipeline_config

##### Configuring the pipeline_config for our model

In [72]:
pipeline_config.model.ssd.num_classes=2
pipeline_config.train_config.batch_size=6
pipeline_config.train_config.fine_tune_checkpoint=pretrained_model_path+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type='detection'
pipeline_config.train_input_reader.label_map_path=annotation_path+'/labelmap.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:]=[annotation_path+'/train.record']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:]=[annotation_path+'/test.record']
pipeline_config.eval_input_reader[0].label_map_path=annotation_path+'/labelmap.pbtxt'

##### saving the configured config file

In [73]:
new_config=text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(config_path,'w') as f:
    f.write(new_config)

In [16]:
%pwd

'C:\\Users\\ankit\\Py-world\\Computer Vision\\TFOD\\RealTimeObjectDetection'

##### Training the model

###### pass the api model path, config file path , number of training steps and run  the above command in terminal to train then model

In [52]:
print('python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=1000')

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=1000


##### Loading the trained model pipeline.config file and last checkpoint on which the model was trained

In [7]:
#: load the model
config=config_util.get_configs_from_pipeline_file(config_path)
detection_model=model_builder.build(model_config=config['model'],is_training=False)
#: load the checkpoints
chk=tf.compat.v2.train.Checkpoint(model=detection_model)
chk.restore(os.path.join(checkpoint_path,'ckpt-7')).expect_partial()

##### Creating a function for detection

In [8]:
@tf.function
def detect(image):
    image,shape=detection_model.preprocess(image)  #: resizing the image, normalizing pixel values, and other preprocessing steps
    prediction=detection_model.predict(image,shape)    #: make the prediction (class score, bounding boxes and other relevant infomations)
    detections=detection_model.postprocess(prediction,shape) #: : filtering out the low confidence predictions, applying non maximum suppression
    return detections

##### Categoryindex

In [9]:
category_index=label_map_util.create_category_index_from_labelmap(annotation_path+'/labelmap.pbtxt')

##### Detection in RealTime

In [10]:
def mask_detection():
    cap = cv2.VideoCapture('./test video 6.mp4')
    # width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    try:
        while True:
            rat,frame=cap.read()
            # frame=cv2.imread('./stock-photo-bearded-man-standing-blue-background-tank-top-pants-looking-away.jpg')
            image_np = np.array(frame)
            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect(input_tensor)
            # print('detections are: ',detections)
            last_detect=detections
            num_detections = int(detections.pop('num_detections')) 
            detections = {key: value[0, :num_detections].numpy()  #: trimming the detection and taking first num_detection values from each key
                          for key, value in detections.items()}
            detections['num_detections'] = num_detections
        
            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
            label_id_offset = 1
            image_np_with_detections = image_np.copy()
        
            viz_util.visualize_boxes_and_labels_on_image_array(
                        image_np_with_detections,
                        detections['detection_boxes'],
                        detections['detection_classes']+label_id_offset,
                        detections['detection_scores'],
                        category_index,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw=5,
                        min_score_thresh=.5,
                        agnostic_mode=False)
        
            cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except Exception as e:
        print(e)
        # cap.release()
        cv2.destroyAllWindows()

    
    # cap.release()
    cv2.destroyAllWindows()
    
    

In [ ]:
mask_detection()